<a href="https://colab.research.google.com/github/jlonge4/gen_ai_utils/blob/main/vllm_vs_deepspeed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flash-attn deepspeed vllm

In [2]:
#release cuda memory using torch clear cache
import torch
torch.cuda.empty_cache()

In [3]:
context = """Instruction tuning large language models (LLMs) using machine-generated
instruction-following data has been shown to improve zero-shot capabilities on
new tasks, but the idea is less explored in the multimodal field. We present the
first attempt to use language-only GPT-4 to generate multimodal language-image
instruction-following data. By instruction tuning on such generated data, we introduce LLaVA: Large Language and Vision Assistant, an end-to-end trained
large multimodal model that connects a vision encoder and an LLM for generalpurpose visual and language understanding. To facilitate future research on visual
instruction following, we construct two evaluation benchmarks with diverse and
challenging application-oriented tasks. Our experiments show that LLaVA demonstrates impressive multimodal chat abilities, sometimes exhibiting the behaviors
of multimodal GPT-4 on unseen images/instructions, and yields a 85.1% relative score compared with GPT-4 on a synthetic multimodal instruction-following
dataset. When fine-tuned on Science QA, the synergy of LLaVA and GPT-4
achieves a new state-of-the-art accuracy of 92.53%. We make GPT-4 generated
visual instruction tuning data, our model, and code publicly available.
1 Introduction
Humans interact with the world through many channels such as vision and language, as each
individual channel has a unique advantage in representing and communicating certain concepts, and
thus facilitates a better understanding of the world. One of the core aspirations in artificial intelligence
is to develop a general-purpose assistant that can effectively follow multi-modal vision-and-language
instructions, aligned with human intent to complete various real-world tasks in the wild [4, 27, 26].
To this end, the community has witnessed an emergent interest in developing language-augmented
foundation vision models [27, 16], with strong capabilities in open-world visual understanding
such as classification [40, 21, 57, 54, 39], detection [29, 62, 33], segmentation [25, 63, 58] and
captioning [50, 28], as well as visual generation and editing [42, 43, 56, 15, 44, 30]. We refer readers
to the Computer Vision in the Wild reading list for a more up-to-date literature compilation [12]. In
this line of work, each task is solved independently by one single large vision model, with the task
instruction implicitly considered in the model design. Further, language is only utilized to describe
the image content. While this allows language to play an important role in mapping visual signals to
language semantics—a common channel for human communication, it leads to models that usually
have a fixed interface with limited interactivity and adaptability to the user’s instructions.
Large language models (LLM), on the other hand, have shown that language can play a wider
role: a universal interface for a general-purpose assistant, where various task instructions can be
explicitly represented in language and guide the end-to-end trained neural assistant to switch to the
task of interest to solve it. For example, the recent success of ChatGPT [35] and GPT-4 [36] have
demonstrated the power of aligned LLMs in following human instructions, and have stimulated
tremendous interest in developing open-source LLMs. Among them, LLaMA [49] is an opensource LLM that matches the performance of GPT-3. Alpaca [48], Vicuna [9], GPT-4-LLM [38]
37th Conference on Neural Information Processing Systems (NeurIPS 2023).
arXiv:2304.08485v2 [cs.CV] 11 Dec 2023
utilize various machine-generated high-quality instruction-following samples to improve the LLM’s
alignment ability, reporting impressive performance compared with proprietary LLMs. Importantly,
this line of work is text-only.
In this paper, we present visual instruction-tuning, the first attempt to extend instruction-tuning to
the language-image multimodal space, to pave the way towards building a general-purpose visual
assistant. In particular, our paper makes the following contributions:
• Multimodal instruction-following data. One key challenge is the lack of vision-language
instruction-following data. We present a data reformation perspective and pipeline to convert
image-text pairs into an appropriate instruction-following format, using ChatGPT/GPT-4.
• Large multimodal models. We develop a large multimodal model (LMM), by connecting the
open-set visual encoder of CLIP [40] with the language decoder Vicuna [9], and fine-tuning
end-to-end on our generated instructional vision-language data. Our empirical study validates
the effectiveness of using generated data for LMM instruction-tuning, and suggests practical
tips for building a general-purpose instruction-following visual agent. When ensembled with
GPT-4, our approach achieves SoTA on the Science QA [34] multimodal reasoning dataset.
• Multimodal instruction-following benchmark. We present LLaVA-Bench with two challenging
benchmarks, with a diverse selection of paired images, instructions and detailed annotations.
• Open-source. We release the following assets to the public: the generated multimodal instruction
data, the codebase, the model checkpoints, and a visual chat demo.
2 Related Work
Multimodal Instruction-following Agents. In computer vision, existing works that build instructionfollowing agents can be broadly categorized into two classes: (i) End-to-end trained models, which
are separately explored for each specific research topic. For example, the vision-language navigation
task [3, 19] and Habitat [47] require the embodied AI agent to follow natural language instructions
and take a sequence of actions to complete goals in visual environments. In the image editing domain,
given an input image and a written instruction that tells the agent what to do, InstructPix2Pix [6]
edits images by following the human instructions. (ii) A system that coordinates various models
via LangChain [1] / LLMs [35], such as Visual ChatGPT [53], X-GPT [63], MM-REACT [55],
VisProg [18], and ViperGPT [46]. While sharing the same goal in building instruction-following
agents, we focus on developing an end-to-end trained language-vision multimodal model for multiple
tasks.
Instruction Tuning. In the natural language processing (NLP) community, to enable LLMs such
as GPT-3 [7], T5 [41], PaLM [10], and OPT [60] to follow natural language instructions and
complete real-world tasks, researchers have explored methods for LLM instruction-tuning [37, 52, 51],
leading to instruction-tuned counterparts such as InstructGPT [37]/ChatGPT [35], FLAN-T5 [11],
FLAN-PaLM [11], and OPT-IML [22], respectively. It turns out that this simple approach can
effectively improve the zero- and few-shot generalization abilities of LLMs. It is thus natural
to borrow the idea from NLP to computer vision. More broadly, the teacher-student distillation
ideas with foundation models have been studied in other topics such as image classification [14].
Flamingo [2] can be viewed as the GPT-3 moment in the multimodal domain, due to its strong
performance on zero-shot task transfer and in-context-learning. Other LMMs trained on imagetext pairs include BLIP-2 [28], FROMAGe [24], and KOSMOS-1 [20]. PaLM-E [13] is an LMM
for embodied AI. Based on the recent “best” open-source LLM LLaMA, OpenFlamingo [5] and
LLaMA-Adapter [59] are open-source efforts that enable LLaMA to use image inputs, paving the
way to build open-source multimodal LLMs. While these models present promising task transfer
generalization performance, they are not explicitly tuned with vision-language instruction data, and
their performance in multimodal tasks usually falls short compared to language-only tasks. In this
paper, we aim to fill this gap and study its effectiveness. Finally, note that visual instruction tuning
is different from visual prompt tuning [23]: the former aims to improve the model’s instructionfollowing abilities, while the latter aims to improve the parameter-efficiency in model adaptation.
3 GPT-assisted Visual Instruction Data Generation
"""

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from deepspeed.module_inject import HFBertLayerPolicy
import deepspeed

# Model Repository on huggingface.co
model_id='microsoft/phi-2'

# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", trust_remote_code=True)

# init deepspeed inference engine
ds_model = deepspeed.init_inference(
    model=model,      # Transformers models
    mp_size=1,        # Number of GPU
    dtype=torch.half, # dtype of the weights (fp16)
    # injection_policy={"BertLayer" : HFBertLayerPolicy}, # replace BertLayer with DS HFBertLayerPolicy
    replace_method="auto", # Lets DS autmatically identify the layer to replace
    replace_with_kernel_inject=True, # replace the model with the kernel injector
    max_tokens=2048,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[2024-03-30 03:07:14,253] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.14.0, git-hash=unknown, git-branch=unknown
[2024-03-30 03:07:14,256] [WARNING] [config_utils.py:69:_process_deprecated_field] Config parameter replace_method is deprecated. This parameter is no longer needed, please remove from your call to DeepSpeed-inference
[2024-03-30 03:07:14,257] [WARNING] [config_utils.py:69:_process_deprecated_field] Config parameter mp_size is deprecated use tensor_parallel.tp_size instead
[2024-03-30 03:07:14,266] [INFO] [logging.py:96:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


In [7]:
ds_model

InferenceEngine(
  (module): PhiForCausalLM(
    (model): PhiModel(
      (embed_tokens): Embedding(51200, 2560)
      (embed_dropout): Dropout(p=0.0, inplace=False)
      (layers): ModuleList(
        (0-31): 32 x PhiDecoderLayer(
          (self_attn): PhiAttention(
            (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (dense): Linear(in_features=2560, out_features=2560, bias=True)
            (rotary_emb): PhiRotaryEmbedding()
          )
          (mlp): PhiMLP(
            (activation_fn): NewGELUActivation()
            (fc1): Linear(in_features=2560, out_features=10240, bias=True)
            (fc2): Linear(in_features=10240, out_features=2560, bias=True)
          )
          (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
          (resid_dropout): Dropout(p=0.1, inp

In [17]:
question = 'how do Humans interact with the world?'

In [14]:
ds_clf = pipeline("text-generation", model=ds_model, tokenizer=tokenizer,device=0, max_length=2000)

# Test pipeline
example = f'Instruct:Based on the context {context}, answer the question {question}. Output:'

The model 'InferenceEngine' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalL

7.7GB GPU / Inference in 3.14s

In [15]:
%%time
results = ds_clf(example)
print(results[0]['generated_text'].split('Output:')[1])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Humans interact with the world through many channels such as vision and language, as each
individual channel has a unique advantage in representing and communicating certain concepts, and
thus facilitates a better understanding of the world.
3.1.
Human Interaction with the World
Humans interact with the world through many channels such as vision and language
CPU times: user 3.13 s, sys: 23.8 ms, total: 3.15 s
Wall time: 3.14 s


In [18]:
import torch
torch.cuda.empty_cache()

In [31]:
sampling_params = SamplingParams(temperature=0.05, top_p=0.95, max_tokens=500)

In [ ]:
from vllm import LLM, SamplingParams
llm = LLM(model="microsoft/phi-2")

15.9GB GPU RAM / 1.46s

In [35]:
completion = llm.generate(f'Instruct:Based on the context {context}, answer the question {question}. Output:', sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.46s/it]


In [36]:
completion[0].outputs[0].text

'\nHumans interact with the world through many channels such as vision and language, as each\nindividual channel has a unique advantage in representing and communicating certain concepts, and\nthus facilitates a better understanding of the world.\n3.1. The Importance of Visual-Language Interaction\nHumans interact with the world through many channels such as vision and language, as each\nindividual channel has a unique advantage in representing and communicating certain concepts, and\nthus facilitates a better understanding of the world.\n3.2. The Challenge of Visual-Language Interaction\nHumans interact with the'